In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [2]:
train_data, test_data = imdb["train"], imdb["test"]

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [3]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [4]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a <OOV> <OOV> <OOV> when the rest of the world can go <OOV> with its own business as you <OOV> into a big <OOV> <OOV> and <OOV> for a couple of hours wonderful performances from <OOV> and <OOV> <OOV> as always <OOV> <OOV> the plot along there are no <OOV> to <OOV> no <OOV> <OOV> just a <OOV> and <OOV> <OOV> through new york life at its best a family film in every sense and one that <OOV> the <OOV> it <OOV>
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               983552    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,000,065
Trainable params: 1,000,065
Non-trainable params: 0
_________________________________________________________________


In [6]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 13s 14ms/step - loss: 0.5825 - accuracy: 0.6549 - val_loss: 0.3949 - val_accuracy: 0.8215
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3499 - accuracy: 0.8475 - val_loss: 0.4191 - val_accuracy: 0.8110
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2387 - accuracy: 0.9060 - val_loss: 0.5053 - val_accuracy: 0.7889
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1122 - accuracy: 0.9646 - val_loss: 0.7314 - val_accuracy: 0.7732
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 0.0396 - accuracy: 0.9921 - val_loss: 0.9924 - val_accuracy: 0.7760
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0138 - accuracy: 0.9983 - val_loss: 1.2215 - val_accuracy: 0.7743
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0133 - accuracy: 0.9969 - val_loss: 1.2942 - val_accuracy: 0.769

In [7]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [8]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [9]:
model.evaluate(testing_padded, testing_labels_final)

782/782 [==============================] - 3s 4ms/step - loss: 1.5918 - accuracy: 0.7749


[1.5918126106262207, 0.7748799920082092]

In [10]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "acc")
plot_graphs(history, "loss")

NameError: name 'history' is not defined